In [1]:
import pandas as pd
import numpy as np

def calculate_decay_weight(days_old, half_life=7):
    """
    Calculate exponential decay weight based on age.
    
    Parameters:
    - days_old: Number of days since recommendation
    - half_life: Number of days for weight to decay to 50% (default: 7 days)
    
    Returns weight between 0 and 1
    """
    return np.exp(-np.log(2) * days_old / half_life)

# Load the data
df = pd.read_csv('sp500_ibes_recommendations_2015_2024.csv')

# Convert date columns to datetime
df['anndats'] = pd.to_datetime(df['anndats'])
df['ireccd'] = df['ireccd']-3

print("Processing analyst recommendations with exponential decay...")
print(f"Total recommendations: {len(df)}")

# Parameters for decay
half_life = 7  # Days for weight to decay to 50%
lookback_days = 30  # Consider recommendations from past 30 days

# Get all unique month-ends in the data
all_months = pd.date_range(
    start=df['anndats'].min().replace(day=1),
    end=df['anndats'].max() + pd.offsets.MonthEnd(0),
    freq='M'
)

print(f"Using exponential decay with half_life={half_life} days, lookback={lookback_days} days")

# Storage for monthly aggregations
monthly_results = []

# Process each month-end
for month_end in all_months:
    # Get lookback window
    lookback_start = month_end - pd.Timedelta(days=lookback_days)
    
    # Filter recommendations within lookback window
    month_recs = df[(df['anndats'] > lookback_start) & (df['anndats'] <= month_end)].copy()
    
    if len(month_recs) == 0:
        continue
    
    # Calculate days old for each recommendation (relative to month end)
    month_recs['days_old'] = (month_end - month_recs['anndats']).dt.days
    
    # Calculate decay weights
    month_recs['decay_weight'] = month_recs['days_old'].apply(
        lambda x: calculate_decay_weight(x, half_life=half_life)
    )
    
    # Group by permno and calculate weighted mean
    grouped = month_recs.groupby(['permno', 'ticker_crsp'])
    
    for (permno, ticker), group in grouped:
        # Calculate weighted mean recommendation
        weighted_mean = (group['ireccd'] * group['decay_weight']).sum()
        
        # Calculate unweighted mean for comparison
        unweighted_mean = group['ireccd'].mean()
        
        # Effective count (sum of weights)
        effective_count = group['decay_weight'].sum()
        
        monthly_results.append({
            'permno': permno,
            'ticker': ticker,
            'date': month_end,
            'weighted_mean_recommendation': weighted_mean,
            'unweighted_mean_recommendation': unweighted_mean,
            'num_recommendations': len(group),
            'effective_recommendation_count': effective_count
        })

# Create DataFrame
monthly_mean = pd.DataFrame(monthly_results)

print(f"\nGenerated {len(monthly_mean)} monthly records")

# Sort by permno and date to calculate changes
monthly_mean = monthly_mean.sort_values(['permno', 'date']).reset_index(drop=True)

# Calculate the change in weighted mean recommendation from previous month
monthly_mean['recommendation_change'] = monthly_mean.groupby('permno')['weighted_mean_recommendation'].diff()

# Also calculate unweighted change for comparison
monthly_mean['unweighted_change'] = monthly_mean.groupby('permno')['unweighted_mean_recommendation'].diff()

print("\nCalculating significant changes...")

# Filter for stocks with absolute change >= 1
significant_changes = monthly_mean[monthly_mean['recommendation_change'].abs() >= 0.5].copy()

# Reorder columns
significant_changes = significant_changes[[
    'permno', 'date', 'ticker', 
    'weighted_mean_recommendation', 'recommendation_change',
    'unweighted_mean_recommendation', 'unweighted_change',
    'num_recommendations', 'effective_recommendation_count'
]]

# Sort by date and absolute change (largest changes first)
significant_changes = significant_changes.sort_values(
    ['date', 'recommendation_change'], 
    key=lambda x: x if x.name != 'recommendation_change' else abs(x),
    ascending=[True, False]
).reset_index(drop=True)

# Display results
print("\n" + "="*80)
print("Significant Recommendation Changes (±1 or more) - WITH EXPONENTIAL DECAY")
print("="*80)
print(significant_changes.head(20))
print(f"\nShape: {significant_changes.shape}")
print(f"\nDate range: {significant_changes['date'].min()} to {significant_changes['date'].max()}")
print(f"Total significant changes: {len(significant_changes)}")
print(f"Unique stocks with significant changes: {significant_changes['permno'].nunique()}")

# Summary statistics
print("\n" + "="*80)
print("Summary Statistics (Weighted):")
print(f"Mean absolute change: {significant_changes['recommendation_change'].abs().mean():.3f}")
print(f"Max upgrade (negative change): {significant_changes['recommendation_change'].min():.3f}")
print(f"Max downgrade (positive change): {significant_changes['recommendation_change'].max():.3f}")

# Count upgrades vs downgrades
upgrades = (significant_changes['recommendation_change'] <= -0.5).sum()
downgrades = (significant_changes['recommendation_change'] >= 0.5).sum()
print(f"\nUpgrades (change <= -0.5): {upgrades}")
print(f"Downgrades (change >= 0.5): {downgrades}")

# Compare weighted vs unweighted
print("\n" + "="*80)
print("Comparison: Weighted vs Unweighted Changes")
print(f"Correlation between weighted and unweighted changes: "
      f"{significant_changes['recommendation_change'].corr(significant_changes['unweighted_change']):.3f}")

# Show sample where weighted differs from unweighted
sample_diff = significant_changes.copy()
sample_diff['abs_difference'] = abs(sample_diff['recommendation_change'] - sample_diff['unweighted_change'])
sample_diff = sample_diff.nlargest(5, 'abs_difference')

print("\nTop 5 cases where weighted differs most from unweighted:")
print(sample_diff[['permno', 'ticker', 'date', 'weighted_mean_recommendation', 
                   'unweighted_mean_recommendation', 'recommendation_change', 
                   'unweighted_change']].to_string(index=False))

# Distribution of changes
print("\n" + "="*80)
print("Distribution of Weighted Recommendation Changes:")
print(significant_changes['recommendation_change'].describe())

# # Save results
# print("\n" + "="*80)
# print("Saving results...")

# # Save significant changes
# significant_changes.to_csv('significant_recommendation_changes_decay.csv', index=False)
# print("Saved: significant_recommendation_changes_decay.csv")

# Save full monthly means with changes
monthly_mean_output = monthly_mean[[
    'permno', 'date', 'ticker',
    'weighted_mean_recommendation', 'recommendation_change',
    'unweighted_mean_recommendation', 'unweighted_change',
    'num_recommendations', 'effective_recommendation_count'
]]
monthly_mean_output.to_csv('monthly_mean_recommendations_decay.csv', index=False)
print("Saved: monthly_mean_recommendations_decay.csv")

print("\n" + "="*80)
print("SUMMARY")
print("="*80)
print(f"Half-life: {half_life} days")
print(f"Lookback window: {lookback_days} days")
print(f"Total monthly records: {len(monthly_mean)}")
print(f"Significant changes (±1): {len(significant_changes)}")
print(f"Unique stocks tracked: {monthly_mean['permno'].nunique()}")

Processing analyst recommendations with exponential decay...
Total recommendations: 61244
Using exponential decay with half_life=7 days, lookback=30 days


/var/folders/w3/6myr5tjs6ls3zn1g3ywbc29r0000gn/T/ipykernel_54137/3020687687.py:31: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  all_months = pd.date_range(



Generated 37527 monthly records

Calculating significant changes...

Significant Recommendation Changes (±1 or more) - WITH EXPONENTIAL DECAY
    permno       date ticker  weighted_mean_recommendation  \
0    85035 2015-01-31   QRVO                     -4.481718   
1    21186 2015-01-31    MWV                     -3.345805   
2    83435 2015-01-31   YHOO                     -3.289392   
3    89393 2015-01-31   NFLX                     -2.274952   
4    89195 2015-01-31    PFG                     -1.928962   
5    89014 2015-01-31    COL                     -1.760573   
6    86158 2015-01-31   CTSH                     -1.723237   
7    50017 2015-01-31    RRC                     -2.077713   
8    89258 2015-01-31    PRU                     -1.640671   
9    87541 2015-01-31    PKG                     -1.485994   
10   92203 2015-01-31   LULU                     -1.202676   
11   61399 2015-01-31    LOW                     -0.371499   
12   90537 2015-01-31     NE                      1

In [2]:
monthly_mean[monthly_mean['date']=='2020-03-31']

,permno,ticker,date,weighted_mean_recommendation,unweighted_mean_recommendation,num_recommendations,effective_recommendation_count,recommendation_change,unweighted_change
46,10104,ORCL,2020-03-31,-0.631912,-1.333333,3,0.518696,-0.493900,-0.333333
188,10138,TROW,2020-03-31,0.500000,1.000000,1,0.500000,0.556608,2.000000
227,10145,ACD,2020-03-31,-0.410168,-0.333333,3,1.863030,0.042694,0.666667
314,10516,ADM,2020-03-31,0.000000,0.000000,1,0.056608,-0.226431,-2.000000
369,10696,FI,2020-03-31,-1.101349,-1.000000,3,1.101349,-1.101349,-1.000000
...,...,...,...,...,...,...,...,...,...
37146,93132,FTNT,2020-03-31,0.000000,0.000000,2,0.922950,1.031267,1.000000
37244,93159,ESV,2020-03-31,1.826606,0.888889,9,2.150070,1.531013,0.555556
37273,93246,GNRC,2020-03-31,-1.219014,-2.000000,1,0.609507,-0.313290,0.000000
37349,93422,QEP,2020-03-31,0.905724,0.250000,4,1.457956,0.905724,0.250000
